> #  Data Analysis & Visualization &nbsp;- Designed by Michael Cheng

## Project Problem Statement - Auto-mpg Analysis

### Background

**Context**

The shifting market conditions, globalization, cost pressure, and volatility are leading to a change in the automobile market landscape. The emergence of data, in conjunction with machine learning in automobile companies, has paved a way that is helping bring operational and business transformations.

The automobile market is vast and diverse, with numerous vehicle categories being manufactured and sold with varying configurations of attributes such as displacement, horsepower, and acceleration. We aim to find combinations of these features that can clearly distinguish certain groups of automobiles from others through this analysis, as this will inform other downstream processes for any organization aiming to sell each group of vehicles to a slightly different target audience.

You are a Data Scientist at SecondLife which is a leading used car dealership with numerous outlets across the US. Recently, <span style="background-color: yellow">they have started shifting their focus to vintage cars and have been diligently collecting data about all the vintage cars they have sold over the years. The Director of Operations at SecondLife wants to leverage the data to extract insights about the cars and find different groups of vintage cars to target the audience more efficiently.</span> [1]

**Objective**

<span style="background-color: yellow">The objective of this problem is to explore the data, extract meaningful insights, and find different groups of vehicles in the data by using dimensionality reduction techniques like PCA and t-SNE.</span>

**Data Description:**
There are 8 variables in the dataset:

* mpg: miles per gallon

* cyl: number of cylinders

* disp: engine displacement (cu. inches) or engine size

* hp: horsepower
  
* wt: vehicle weight (lbs.)
  
* acc: time taken to accelerate from 0 to 60 mph (sec.)

* yr: model year
  
* car name: car model name

### Import Libraries & Load the data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Importing PCA and t-SNE
from sklearn.decomposition import PCA

from sklearn.manifold import TSNE

# Summary Tools
from summarytools import dfSummary
data = pd.read_csv("auto-mpg.csv")

### DataPreprocessing

In [ ]:
# Copy of data
df = data.copy()

# Overview of data
print(df.head())
df.info()
dfSummary(df)

#### The overview above shows: 

1. There are no duplicates or missing values.
2. The average model year is 1976, with a relatively low spread (3.7 years).
3. The minimum year is 1970, the median is 1976, and the maximum is 1982, showing a symmetric spread around the median.
4. With an Inter-Quartile Range (IQR) at 6.0; this suggests the middle 50% of model years span 6 years (i.e. from 1973 to 1979, depending on the exact quartiles).
5. Coefficient of Variation (CV) shows a relative variability of 20.6%; this shows a moderately high standard deviation at about 20.6% of the mean, indicating reasonable variability in model years
6. The dataset represents a vehicle's **Performance Features** in the following:
   
- mpg
- cylinders
- displacement
- horsepower
- weight
- acceleration

In [ ]:
# Objects Columns: Review 'car name'
df['car name'].unique()

#### Decision Point: 
- Because the firm has *"recently started"* [[1]](#Background_1) shifting their attention to vintage vehicles, this dataset will contain vintage *and non-vintage* vehicles. 
- Although "vintage" may literally be designated to a particular year rather than the particular make/model of a vehicle, there is also a significant factor from *cultural perception* that plays into a vehicle's **vintage value**.
- Since the firm desires to "leverage the data to extract insights about the cars and find different groups of vintage cars to target the audience more efficiently" [[1]](#Background_1), 'car name' will be considered along with other features. 

In [ ]:
# Objects Columns: Review 'horsepower' 
df['horsepower'].unique()

In [ ]:
# Undefined value "?" occurence
print("Instances of ? in 'horsepower'")
df['horsepower'].value_counts()['?']

#### Decision Point: 
- "Horsepower" should be converted to a numeric data type for meaningful analysis and visualization
- "?" values determination: Use Regression (generalization based on global patterns) rather than KNN accounting for variabilities based on local patterns

In [ ]:
# Use Regression to predict "?" horsepower values

from sklearn.linear_model import LinearRegression

# Replace '?' with NaN and convert to numeric
df['horsepower'] = pd.to_numeric(df['horsepower'].replace('?', np.nan), errors='coerce')

# Split rows with and without missing horsepower
df_missing_hp = df[df['horsepower'].isna()]
df_non_missing_hp = df[~df['horsepower'].isna()]

# Features and target for non-missing rows
X = df_non_missing_hp[['mpg', 'cylinders', 'displacement', 'weight', 'acceleration', 'model year']]
y = df_non_missing_hp['horsepower']

# Train a regression model
model = LinearRegression()
model.fit(X, y)

# Predict missing horsepower
X_missing = df_missing_hp[['mpg', 'cylinders', 'displacement', 'weight', 'acceleration', 'model year']]
df.loc[df['horsepower'].isna(), 'horsepower'] = model.predict(X_missing)

# Display updated DataFrame
df


In [ ]:
# Check for null values
df['horsepower'].isna().sum() 

In [ ]:
# Review unique values
df['horsepower'].unique()

##### Decision Point
There are a few instances of trailing decimals that will be distracting for display purposes. This feature, however, will require domain experts to validate. Moreover, Vintage Classification is not primarily determined by **horsepower**, or any other *performance features* (see [above](#The_overview_above_shows:)). Thus, it seems better to leave these records as flagged entries by virtue of their trailing decimals, and allow domain experts to review. 

In [ ]:
# Create an index column for visualization traceability
df['index'] = df.index

In [ ]:
# Review preprocessed data
df.info()

#### Observations: 
* Make and Model, as discused above, can be sigificant in the determination of a vehicle's vintage classification
* Most (89%) of the 'car name' feature is singular (designated as "Other" in frequency distribution)[[2]](#DataPreprocessing_1), which can be the very reason a vehicle could be classifed as "vintage"
* Therefore, these 'car name' labels should be further reviewed, and disaggregated if possible


### Exploratory Data Analysis and Hypothesis Generation

#### t-SNE_preview

In [ ]:
# t-SNE analysis to review car name

from sklearn.manifold import TSNE
import plotly.express as px
import warnings

# Future warning suppressed
warnings.filterwarnings('ignore', category=FutureWarning)


# Prepare t-SNE data
features = df.drop('car name', axis=1)  
tsne = TSNE(n_components=2, random_state=42, perplexity=min(30, len(features)-1))
features_tsne = tsne.fit_transform(features)

# Create DataFrame
tsne_df = pd.DataFrame(features_tsne, columns=['t-SNE 1', 't-SNE 2'])
tsne_df['car name'] = df['car name']

# Plotly visualization with tuple for color parameter
fig = px.scatter(tsne_df, x='t-SNE 1', y='t-SNE 2', 
                 color='car name', 
                 hover_data=['car name'],
                 title='t-SNE Visualization of car name')

fig.update_layout(
   title='t-SNE Visualization of Car Name',
   autosize=False,
   width=800,
   height=300,
   xaxis=dict(
       title='',
       showticklabels=False, 
       showgrid=False, 
       zeroline=False
   ),
   yaxis=dict(
       title='',
       showticklabels=False, 
       showgrid=False, 
       zeroline=False
   )
)

fig.show()

In [ ]:
# t-SNE to review only "Other" in car name

# Identify car names with a frequency of 1
singular_names = df['car name'].value_counts()[df['car name'].value_counts() == 1].index

# Filter the dataset to include only those car names
singular_df = df[df['car name'].isin(singular_names)]

# Drop 'car name' for t-SNE, use other features for singular entries only
features_singular = singular_df.drop('car name', axis=1)

# Apply t-SNE to singular entries
tsne_singular = TSNE(n_components=2, random_state=42, perplexity=min(30, len(features_singular)-1))
features_tsne_singular = tsne_singular.fit_transform(features_singular)

# Create DataFrame for Plotly
tsne_singular_df = pd.DataFrame(features_tsne_singular, columns=['t-SNE 1', 't-SNE 2'])
tsne_singular_df['car name'] = singular_df['car name'].values

import plotly.express as px

# Plotly visualization
fig = px.scatter(tsne_singular_df, x='t-SNE 1', y='t-SNE 2', 
                 color='car name', 
                 hover_data=['car name'],  # Hover over points to see car name
                 title='t-SNE Visualization of Singular Frequency Car Names')

# Update layout for cleaner visualization
fig.update_layout(
    autosize=False,
    width=800,
    height=300,
    xaxis=dict(title='',showticklabels=False, showgrid=False, zeroline=False),
    yaxis=dict(title='',showticklabels=False, showgrid=False, zeroline=False),
    title='t-SNE Visualization of "Other" (Rare) Car Names'
)

fig.show()



#### Observations:
The data points are widely scattered throughout the t-SNE plots, indicating limited clear structure or clustering patterns, which suggests the need for additional feature refinement or alternative dimensionality reduction techniques to better capture relationships.

* Although t-SNE visualized clusters by association with vehicle performance features, the display of the car name labels themselves show that:
  1. There are inconsistencies with how a same make of vehicle, i.e. Volkswagon, is designated as "VW"
  2. Vehicles' car name meaning can be improved by extracting the first term in the field
  3. Further grouping of car name may be performed using fuzzy matching as needed

#### Feature Engineering: Car Brand review and clean-up

In [ ]:
# Extract the first word from 'car name'
df['car_brand'] = df['car name'].str.split().str[0]

# View the unique brands
print(df['car_brand'].unique())


##### Observation
* Many misspelled words, so fuzzy matching may be helpful

In [ ]:
# Selective review of extracted car brands
df[df['car_brand'] == 'hi']

##### Observations
* Internet search results for "1970 1200d" revealed that this is a vintage International Harvester: "Overall, the 1970 International Harvester 1200D is a rare and sought-after vintage pickup truck, prized for its ruggedness, reliability, and nostalgic appeal."
* This confirms that 'car name' could be valuable in the Vintage Classification of the dataset

##### Decision Point
* "hi" will be renamed "harvester", and the car name will remain as it was entered in the dataset

In [ ]:
# Rename "hi" to "harvester"
df.loc[df['car_brand'] == 'hi', 'car_brand'] = 'harvester'

In [ ]:
# Confirm update 
df[df['car_brand'] == 'harvester']

#### Clean-up car brand with mapping

In [ ]:
# Cleanup typos with mapping of known inconsistencies

brand_mapping = {
    'chevy': 'chevrolet',
    'chevroelt': 'chevrolet',
    'vw': 'volkswagen',
    'vokswagen': 'volkswagen',
    'toyouta': 'toyota',
    'maxda': 'mazda',
    'mercedes': 'mercedes-benz'
}

# Apply the mapping to standardize the brands
df['car_brand'] = df['car_brand'].replace(brand_mapping)


In [ ]:
# Selective review of extracted car brands (capri vs ford vs mercury): Capri produced by Mercury, owned by Ford
print(df[df['car_brand'] == 'ford'][['model year', 'car name', 'index', 'car_brand']], "\n")
print(df[df['car_brand'] == 'mercury'][['model year', 'car name', 'index', 'car_brand']], "\n")
print(df[df['car_brand'] == 'capri'][['model year', 'car name', 'index', 'car_brand']], "\n")

##### Decision Point
* "capri ii", identified with a "capri" car brand would be better classified as a "mercury" car brand when queried with other mercury capri vehicles in the data set
* Thus, "mercury" will replace its brand, while "capri ii" will remain as its car name

In [ ]:
# Rename "capri" car brand to "mercury"
df.loc[df['car_brand'] == 'capri', 'car_brand'] = 'mercury'

# Confirm updated car brand
df[df['car_brand'] == 'mercury']

##### Observation
* The "capri ii" is now categorized under the "mercury" car brand. However, further preprocessing of the car name field to extract explicit models (e.g., "car_model") will enhance subsequent reviews.

In [ ]:
# Review unique brands
print(df['car_brand'].unique())

##### Observation
* The list of car brands are now consistent and succinct
* A fuzzy match will ensure there are no surprises between car_brand and car name

In [ ]:
from fuzzywuzzy import fuzz

# Function to calculate fuzzy match score
def calculate_fuzzy_score(row):
    return fuzz.ratio(row['car name'], row['car_brand'])

# Create a new DataFrame for testing
df_fuzzy_test = df[['car name', 'car_brand']].copy()
df_fuzzy_test['fuzzy_score'] = df_fuzzy_test.apply(calculate_fuzzy_score, axis=1)

# View potential mismatches
df_fuzzy_test_sorted = df_fuzzy_test.sort_values(by='fuzzy_score', ascending=True)

# Show mismatches for inspection
print(df_fuzzy_test_sorted.head(20))  # Display the lowest scores


##### Observation
* The fuzzy match shows that car_brand can be reliabily used for visualization and clustering analysis

In [ ]:
# Review count of unique car brands

df['car_brand'].value_counts()

In [ ]:
# Review other single-record car brands
print(df[df['car_brand'] == 'nissan'], "\n")
print("---")
print("\n",df[df['car_brand'] == 'triumph'])


##### Findings
While Triumph is a single-record brand, similar to International Harvester, the Nissan Stanza is owned by Datsun and should be reviewed to determine whether it should be reassigned to this brand.

In [ ]:
df[df['car_brand'] == 'datsun']

##### Decision Point
There are no "Stanza" models in the above 'datsun" filtered car brand dataset. According to an internet search, while Nissan vehicles were sold under the Datsun brand until 1983, the Datsun brand was subsequently discontinued, and the Stanza XE was rebranded as the Nissan Stanza XE. Therefore, the single "Nissan" record will remain unchanged in this analysis.

#### t-SNE_review

In [ ]:
from sklearn.preprocessing import LabelEncoder
import plotly.express as px
import warnings

# Suppress selected Future Warning
warnings.filterwarnings(
    "ignore",
    message="When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas.*",
    category=FutureWarning
)


# Encode categorical car brands
le = LabelEncoder()
df['car_brand_encoded'] = le.fit_transform(df['car_brand'])

# Drop unnecessary columns and prepare features
features = df.drop(['car name', 'car_brand'], axis=1)

# Apply t-SNE
tsne = TSNE(n_components=2, random_state=42, perplexity=min(30, len(features)-1))
features_tsne = tsne.fit_transform(features)

# Create a DataFrame for visualization
tsne_df = pd.DataFrame(features_tsne, columns=['t-SNE 1', 't-SNE 2'])
tsne_df['car_brand'] = df['car_brand']
tsne_df['car_brand_encoded'] = df['car_brand_encoded']

# Plot the t-SNE visualization
fig = px.scatter(tsne_df, x='t-SNE 1', y='t-SNE 2', 
                 color='car_brand', 
                 hover_data=['car_brand'])

# Update layout for cleaner visualization
fig.update_layout(
    autosize=False,
    width=800,
    height=300,
    xaxis=dict(title='',showticklabels=False, showgrid=False, zeroline=False),
    yaxis=dict(title='',showticklabels=False, showgrid=False, zeroline=False),
    title='t-SNE Visualization of Car Brands'
    )

fig.show()

#### Observation
Clustering based on car brand and vintage classification still seem to be obscured by performance features. It seems reasonable therefore that *removing* these features will yield more coherent groupings (as it pertains to **Vintage Classification** than applying PCA for dimensionality reduction.

#### Hypothesis: Vintage Classification
* Vintage classification is primarily determined by the interaction between car name and model year.

**Null Hypothesis (𝐻0):**
* The interaction between car name and model year sufficiently explains vintage classification *without* the need for additional feature (i.e. *[performance features](#The_overview_above_shows:)*) transformations such as PCA.

* Implication:
    * Clustering without PCA (using car name and model year along with other original features) will yield clusters of similar quality as clustering after PCA.

    * PCA will not significantly improve cluster separability or performance because the most relevant variance is already captured by car name and model year.

**Alternative Hypothesis (𝐻𝑎):**
* PCA will reveal latent features that significantly improve vintage classification beyond the interaction of car name and model year alone.

* Implication:
    * Clustering after PCA will produce significantly better-defined clusters (e.g., higher silhouette scores, better cluster separability).
    * Other features or combinations of features (e.g., displacement, acceleration, or transformed horsepower) contain important latent information relevant to vintage classification.


### KMeans *without* vs *with Performance Features* vs *PCA*

In [ ]:
# Begin with only car brand and model year for 3-cluster scoring using Silhouette Scores
# "Core": without performance features
# "Full": with performance features
# "PCA": with performance features using PCA

from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import warnings

# Suppress specific KMeans warning
warnings.filterwarnings("ignore", message="KMeans is known to have a memory leak on Windows with MKL")

# Label encode 'car_brand'
le = LabelEncoder()
df['car_brand_encoded'] = le.fit_transform(df['car_brand'])

# Core
# Drop performance features and keep 'car_brand_encoded'
features_core = df[['model year', 'car_brand_encoded']]

# Fit KMeans 
kmeans_core = KMeans(n_clusters=3, random_state=42, n_init=10)
labels_core = kmeans_core.fit_predict(features_core)

# Evaluate cluster quality
silhouette_core = silhouette_score(features_core, labels_core)
print(f'Silhouette Score (Core Features - Null Hypothesis): {silhouette_core}')

# Include all features, testing null hypothesis 
features_full = df[['model year', 'car_brand_encoded', 'weight', 'displacement']]

# Fit KMeans
kmeans_full = KMeans(n_clusters=3, random_state=42, n_init=10)
labels_full = kmeans_full.fit_predict(features_full)

# Evaluate cluster quality
silhouette_full = silhouette_score(features_full, labels_full)
print(f'Silhouette Score (Full Features with Non-Performance Metrics): {silhouette_full}')

# Compare alternative hypothesis with PCA + KMeans
from sklearn.decomposition import PCA

# Apply PCA to features before running KMeans
pca = PCA(n_components=2)  # Reduce to 2 dimensions
#features_pca = pca.fit_transform(features_core) Silhouette Score (PCA Features - Alternative Hypothesis): 0.4035467744569793
features_pca = pca.fit_transform(features_full)

# Fit KMeans on PCA-reduced features
kmeans_pca = KMeans(n_clusters=3, random_state=42, n_init=10)
labels_pca = kmeans_pca.fit_predict(features_pca)

# Evaluate cluster quality
silhouette_pca = silhouette_score(features_pca, labels_pca)
print(f'Silhouette Score (PCA Features - Alternative Hypothesis): {silhouette_pca}')


##### Silhouette Score

* Evaluation of the *quality of clustering* as represented by the Silhouette Score: 

    1. Cluster Validity: A higher score indicates that clusters are well-formed and distinct.
    2. Optimal Number of Clusters: The silhouette score can help reveal and identify the ideal number of clusters.
    3. Cluster Interpretability: Assess whether the clustering reflects meaningful patterns in the data (e.g., does it align with vintage classification?).
 
**Findings**
1. Core Features (Null Hypothesis):

* Silhouette Score: 0.4035
* This reflects a moderate clustering quality, suggesting that while model year and car brand alone can form clusters, they lack the nuance to capture fully distinct vintage classifications.

2. Full Features (Non-Performance Metrics):

* Silhouette Score: 0.5872
* A substantial improvement, showing that adding weight and displacement leads to more cohesive and well-separated clusters.
* These features seem to carry valuable information about vehicle design or era-related trends, which help refine vintage classification.

3. PCA on Full Features (Alternative Hypothesis):

* Silhouette Score: 0.5877
* Almost identical to the Full Features model without PCA.
* This suggests that PCA successfully reduces dimensionality while preserving important patterns, but it doesn’t improve cluster separability significantly beyond the raw features.

##### Next Step
* Compare results with scaling

In [ ]:
# Scaling all 3 models to compare

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# Initialize StandardScaler
scaler = StandardScaler()

# Scale Core Features
features_core_scaled = scaler.fit_transform(features_core)

# Scale Full Features
features_full_scaled = scaler.fit_transform(features_full)

# Step 1: PCA on Scaled Full Features
pca = PCA(n_components=3)  
features_pca_scaled = pca.fit_transform(features_full_scaled)

# Step 2: Fit KMeans and Evaluate Silhouette Scores
print('Scaled Silhouette Scoring:')
# Core Features Model
kmeans_core = KMeans(n_clusters=3, random_state=42, n_init=10)
labels_core = kmeans_core.fit_predict(features_core_scaled)
silhouette_core = silhouette_score(features_core_scaled, labels_core)
print(f'Silhouette Score (Core Features - Scaled): {silhouette_core}')

# Full Features Model
kmeans_full = KMeans(n_clusters=3, random_state=42, n_init=10)
labels_full = kmeans_full.fit_predict(features_full_scaled)
silhouette_full = silhouette_score(features_full_scaled, labels_full)
print(f'Silhouette Score (Full Features - Scaled): {silhouette_full}')

# PCA Features Model
kmeans_pca = KMeans(n_clusters=2, random_state=42, n_init=10)
labels_pca = kmeans_pca.fit_predict(features_pca_scaled)
silhouette_pca = silhouette_score(features_pca_scaled, labels_pca)
print(f'Silhouette Score (PCA Features - Scaled): {silhouette_pca}')


##### Finding
* Core Features show the highest silhouette score when compared to full features with and without PCA

In [ ]:
# Visualize the 3 models together

import plotly.express as px
from sklearn.manifold import TSNE
import pandas as pd
import warnings

# Suppress the FutureWarning from Plotly
warnings.filterwarnings("ignore", message="When grouping with a length-1 list-like")

# Decode 'car_brand_encoded' back to original 'car_brand'
df['car_brand'] = le.inverse_transform(df['car_brand_encoded'])

# t-SNE for Core Features
tsne_core = TSNE(n_components=2, random_state=42, perplexity=30)
features_tsne_core = tsne_core.fit_transform(features_core)
tsne_df_core = pd.DataFrame(features_tsne_core, columns=['t-SNE 1', 't-SNE 2'])
tsne_df_core['car brand'] = df.loc[features_core.index, 'car_brand']
tsne_df_core['cluster'] = labels_core
tsne_df_core['model'] = 'Core Features'
tsne_df_core['index'] = features_core.index
tsne_df_core['car name'] = df.loc[features_core.index, 'car name']
tsne_df_core['model year'] = df.loc[features_core.index, 'model year']

# t-SNE for Full Features
tsne_full = TSNE(n_components=2, random_state=42, perplexity=30)
features_tsne_full = tsne_full.fit_transform(features_full)
tsne_df_full = pd.DataFrame(features_tsne_full, columns=['t-SNE 1', 't-SNE 2'])
tsne_df_full['car brand'] = df.loc[features_full.index, 'car_brand']
tsne_df_full['cluster'] = labels_full
tsne_df_full['model'] = 'Full Features'
tsne_df_full['index'] = features_full.index
tsne_df_full['car name'] = df.loc[features_full.index, 'car name']
tsne_df_full['model year'] = df.loc[features_full.index, 'model year']

# t-SNE for PCA Features
tsne_pca = TSNE(n_components=2, random_state=42, perplexity=30)
features_tsne_pca = tsne_pca.fit_transform(features_pca)
tsne_df_pca = pd.DataFrame(features_tsne_pca, columns=['t-SNE 1', 't-SNE 2'])
tsne_df_pca['car brand'] = df.loc[features_full.index, 'car_brand']
tsne_df_pca['cluster'] = labels_pca
tsne_df_pca['model'] = 'PCA Features'
tsne_df_pca['index'] = features_full.index
tsne_df_pca['car name'] = df.loc[features_full.index, 'car name']
tsne_df_pca['model year'] = df.loc[features_full.index, 'model year']

# Combine t-SNE DataFrames
tsne_combined = pd.concat([tsne_df_core, tsne_df_full, tsne_df_pca])

# Plot with facet by model and custom hover
fig = px.scatter(tsne_combined, 
                 x='t-SNE 1', 
                 y='t-SNE 2', 
                 color='cluster', 
                 facet_col='model', 
                 title='t-SNE Comparison of Core vs Full vs Full with PCA')

# Customize hovertemplate
fig.update_traces(
    hovertemplate='<br>Car Brand: %{customdata[0]}<br>Car Name: %{customdata[1]}<br>Model Year: %{customdata[2]}<br>Cluster: %{customdata[3]}<extra></extra>',
    customdata=tsne_combined[['car brand', 'car name', 'model year', 'cluster']].to_numpy()
)

# Suppress t-SNE axis labels for all facets
fig.update_xaxes(title='', showticklabels=False, showgrid=False, zeroline=False, matches='x')
fig.update_yaxes(title='', showticklabels=False, showgrid=False, zeroline=False, matches='y')

# Control dimensions
fig.update_layout(
    autosize=False,
    width=800,
    height=400,
    coloraxis_showscale=False 
)

fig.show()




##### General Observations
* It seems PCA and the Full Features without PCA are very much alike; the shape of these models also resemble the original t-SNE preview and review visualizations
* Core Features model seems to reflect clear, distinct clusters, with apparent separation between clusters; this seem to suggest meaningful differences between data points
* An evaluation of optimal number of clusters should be examined next

#### ElbowPlots

In [ ]:
# Review optimal clusters with Elbow Plot

import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

# Define a function to create the elbow plot
def plot_elbow(features, title):
    inertia_values = []
    cluster_range = range(1, 11)  # Test 1 to 10 clusters

    for k in cluster_range:
        kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
        kmeans.fit(features)
        inertia_values.append(kmeans.inertia_)

    # Plot the elbow plot
    plt.figure(figsize=(4, 3))
    plt.plot(cluster_range, inertia_values, marker='o', linestyle='--')
    plt.title(f'{title}', fontsize=10)
    plt.xlabel('Number of Clusters (k)', fontsize=8)
    plt.ylabel('Inertia (WCSS)', fontsize=8)
    plt.yticks(fontsize=8)
    plt.grid(True)
    plt.show()

# Elbow Plot for Core Features (Only Car Brand and Model Year)
plot_elbow(features_core, 'Core Features (Car Brand + Model Year)')

# Elbow Plot for Full Features (Model Year + Car Brand + Non-Performance Metrics)
plot_elbow(features_full, 'Full Features')

# Elbow Plot for PCA Features (Dimensionality Reduced Features)
plot_elbow(features_pca, 'PCA Features (Full Features Reduced to 2D)')


##### Findings
* All 3 models agree that k = 2 is the optimum number of clusters
* Re-run KMeans where k = 2 for all 3 models
* Also re-run as k=2 to compare scaling for all 3 models

#### Clustering iterations with 2 Clusters

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import warnings

# Suppress specific KMeans warning
warnings.filterwarnings("ignore", message="KMeans is known to have a memory leak on Windows with MKL")

def preprocess_data(df):
    """Encodes categorical variables and prepares data for clustering."""
    # Label encode 'car_brand'
    le = LabelEncoder()
    df['car_brand_encoded'] = le.fit_transform(df['car_brand'])
    return df

def get_features(df):
    """Define the feature sets for clustering."""
    # Core features: minimal set
    features_core = df[['model year', 'car_brand_encoded']]
    # Full features: include additional metrics
    features_full = df[['model year', 'car_brand_encoded', 'weight', 'displacement']]
    return features_core, features_full

def calculate_silhouette_scores(features_core, features_full):
    """Calculates silhouette scores for different feature sets, scaling, and cluster counts."""
    results = []
    scaler = StandardScaler()

    # Core Features: Scaled and Unscaled
    features_core_scaled = scaler.fit_transform(features_core)
    for scaling, features in zip(["Unscaled", "Scaled"], [features_core, features_core_scaled]):
        for k in [2, 3]:
            kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
            labels = kmeans.fit_predict(features)
            silhouette = silhouette_score(features, labels)
            results.append({
                "Feature Set": "Core Features",
                "Scaling": scaling,
                "Clusters": k,
                "Silhouette Score": silhouette
            })

    # Full Features: Scaled and Unscaled
    features_full_scaled = scaler.fit_transform(features_full)
    for scaling, features in zip(["Unscaled", "Scaled"], [features_full, features_full_scaled]):
        for k in [2, 3]:
            kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
            labels = kmeans.fit_predict(features)
            silhouette = silhouette_score(features, labels)
            results.append({
                "Feature Set": "Full Features",
                "Scaling": scaling,
                "Clusters": k,
                "Silhouette Score": silhouette
            })

    # PCA Features: Scaled and Unscaled
    for scaling, features in zip(["Unscaled", "Scaled"], [features_full, features_full_scaled]):
        pca = PCA(n_components=2)
        features_pca = pca.fit_transform(features)
        for k in [2, 3]:
            kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
            labels = kmeans.fit_predict(features_pca)
            silhouette = silhouette_score(features_pca, labels)
            results.append({
                "Feature Set": "PCA Features",
                "Scaling": scaling,
                "Clusters": k,
                "Silhouette Score": silhouette
            })

    return pd.DataFrame(results)

def print_results(df_silhouette):
    """Prints silhouette scores in the desired format."""
    for scaling in ["Unscaled", "Scaled"]:
        print(f"2-Cluster, {scaling} Silhouette Scoring:")
        filtered = df_silhouette[(df_silhouette["Clusters"] == 2) & (df_silhouette["Scaling"] == scaling)]
        for _, row in filtered.iterrows():
            print(f"Silhouette Score ({row['Feature Set']} - {scaling}): {row['Silhouette Score']}")
        print()

def main(df):
    # Step 1: Preprocess Data
    df = preprocess_data(df)

    # Step 2: Get Feature Sets
    features_core, features_full = get_features(df)

    # Step 3: Calculate Silhouette Scores
    df_silhouette = calculate_silhouette_scores(features_core, features_full)

    # Step 4: Print Results
    print_results(df_silhouette)

    # Step 5: Return DataFrame for further analysis
    return df_silhouette

# Run the models: Full vs Core, Scaled vs Unscaled, Produce corresponding silhouette scores
df_silhouette = main(df)


### Statistical Benchmarking and Cross-Context Validation 
* While silhouette scores are meaningful within their immediate contexts for comparison, i.e. 2-cluster vs 3-cluster, scaled vs unscaled, we can more meaningfully compare across models using statistical comparisons of the clustering models
* Coefficient of Variation (CV) and Relative Range can be applied as key metrics to provide a global measure of clustering performance and stability.
* Although all the [elbow plots](#ElbowPlots) of the three models have unanimously identified having 2 clusters as optimal, like the sihouette score, the elbow point is not an *absolute metric*.
    * The elbow plot provides guidance rather than definitive answers in the absolute sense.
    * The results from the elbow plots must therefore be interpreted in conjunction with other metrics like stability and reliability metrics, as well as domain knowledge. 

### Stability and Reliability Metrics

In [ ]:
# Coefficient of Variation: Quantify relative variability across model scenarios

# CV by Feature Set and Scaling (Aggregate Clusters)
cv_results_feature_scaling = df_silhouette.groupby(['Feature Set', 'Scaling'])['Silhouette Score'].apply(
    lambda x: (np.std(x, ddof=1) / np.mean(x)) * 100
)
print("\nCoefficient of Variation (CV) by Feature Set and Scaling:")
print(cv_results_feature_scaling)

# CV by Feature Set and Clusters (Aggregate Scaling)
cv_results_feature_clusters = df_silhouette.groupby(['Feature Set', 'Clusters'])['Silhouette Score'].apply(
    lambda x: (np.std(x, ddof=1) / np.mean(x)) * 100
)
print("\nCoefficient of Variation (CV) by Feature Set and Clusters:")
print(cv_results_feature_clusters)

# CV by Scaling and Clusters (Aggregate Feature Set)
cv_results_scaling_clusters = df_silhouette.groupby(['Scaling', 'Clusters'])['Silhouette Score'].apply(
    lambda x: (np.std(x, ddof=1) / np.mean(x)) * 100
)
print("\nCoefficient of Variation (CV) by Scaling and Clusters:")
print(cv_results_scaling_clusters)

#### Findings: Coefficient of Variation: Stability / Reliability

* The CV analysis demonstrates that Full Features with 3 Clusters is a highly variable model and therefore less reliable for drawing robust insights.
* The **Core Features with 3 Clusters has the lowest CV (3.07%)**:
    * This indicates that the silhouette scores for this configuration are highly consistent across different contexts.
    * It also suggests that the clustering results are stable, making this model more reliable for drawing conclusions.

In [ ]:
# Range and Relative Range by Feature Set and Scaling

range_feature_scaling = df_silhouette.groupby(['Feature Set', 'Scaling'])['Silhouette Score'].agg(
    Range=lambda x: x.max() - x.min(),
    Relative_Range=lambda x: ((x.max() - x.min()) / x.max()) * 100 if x.max() > 0 else np.nan
)

print("\nRange and Relative Range by Feature Set and Scaling:")
print(range_feature_scaling)

# Range and Relative Range by Feature Set and Clusters
range_feature_clusters = df_silhouette.groupby(['Feature Set', 'Clusters'])['Silhouette Score'].agg(
    Range=lambda x: x.max() - x.min(),
    Relative_Range=lambda x: ((x.max() - x.min()) / x.max()) * 100 if x.max() > 0 else np.nan
)

print("\nRange and Relative Range by Feature Set and Clusters:")
print(range_feature_clusters)

# Range and Relative Range by Scaling and Clusters
range_scaling_clusters = df_silhouette.groupby(['Scaling', 'Clusters'])['Silhouette Score'].agg(
    Range=lambda x: x.max() - x.min(),
    Relative_Range=lambda x: ((x.max() - x.min()) / x.max()) * 100 if x.max() > 0 else np.nan
)

print("\nRange and Relative Range by Scaling and Clusters:")
print(range_scaling_clusters)


##### Findings:  Range and Relative Range
* By Scaling: Core Features (Scaled) has the lowest relative range among scaled sets (7.30), indicating *high stability* in scaled data. Core Features (Unscaled) has the lowest relative range among unscaled sets (27.49), showing greater stability in raw form compared to other unscaled features.
* By Clusters: The most stable cluster configuration appears for Core Features with 3 Clusters (4.25), which has the lowest relative range overall.
* By Scaling and Clusters: Unscaled with 2 Clusters has the lowest relative range (12.46), suggesting that unscaled features are more stable with fewer clusters.

Therefore, the most stable overall configuration is Core Features with 3 Clusters. Most stable scaled data is Core Features (Scaled). Most stable unscaled data is Core Features (Unscaled) (27.49). From these, **Core Features with 3 Clusters** is the most stable and reliable configuration overall.

#### Visualization of Findings

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Add Adjusted Silhouette Score (1 - x)
df_silhouette['Adjusted Score'] = 1 - df_silhouette['Silhouette Score']

# Create the figure and subplots
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Subplot 1: Bar Chart
sns.barplot(ax=axes[0], x='Feature Set', y='Adjusted Score', data=df_silhouette, ci=None)
axes[0].set_title('Average Reliability/Stability Score by Feature Set')
axes[0].set_ylabel('Adjusted Silhouette Score (1 - Silhouette Score)')
axes[0].set_xlabel('Feature Set')

# Subplot 2: Heatmap
heatmap_data = df_silhouette.pivot_table(
    index='Feature Set', columns='Clusters', values='Adjusted Score', aggfunc='mean'
)
sns.heatmap(heatmap_data, annot=True, fmt='.2f', cmap='coolwarm', ax=axes[1])
axes[1].set_title('Reliability/Stability Measure: Feature Set x Clusters')
axes[1].set_xlabel('Number of Clusters')
axes[1].set_ylabel('Feature Set')

# Adjust layout for better spacing
plt.tight_layout()

# Show the combined figure
plt.show()


#### Findings: Magnitude of variability relative to the highest silhouette score in each context

* Core Features with 3 Clusters consistently show the *lowest* Relative Range across all groupings, further validating their **stability and reliability**.
* Full Features with 3 Clusters consistently have the *highest* Relative Range, indicating **significant instability**.

### Hypothesis Testing Results, Conclusion & Recommendations

Null Hypothesis (𝐻0):
* The interaction between car name and model year sufficiently explains vintage classification without PCA.

Supporting Evidence:

1. Core Features with 3 Clusters consistently outperforms other feature sets in CV (3.07%) and Relative Range (4.25%).
2. Low silhouette score variability confirms that adding more dimensions with Full Feature Set, or reducing dimensionality using PCA, introduces noise rather than clarity.

Alternative Hypothesis (𝐻𝑎):
* PCA reveals latent features that significantly improve vintage classification.

Counter Evidence:

1. PCA Features with 2 Clusters improve relative variability over most setups, but PCA features introduce higher complexity without outperforming.
2. No configuration after PCA significantly outperforms Core Clusters for Vintage scoring.

**Conclusion & Recommendations**

*Support for Null Hypothesis (𝐻0):*

* Clustering based on car name and model year is sufficient for stable and reliable vintage classification.
* Core Features with 3 Clusters demonstrates lowest variability and most consistent clustering behavior.

*1. Conclusion:* The Core Features with 3 Clusters (see Cluster Diagram and Data Table below) can be leveraged to extract valuable insights about the cars, grouping them to more effectively target the audience. Further analysis with domain experts will be necessary to achieve this.

*2. Recommendations:*
* Start with the simpler Core Features, avoiding unnecessary transformations.
    * PCA may introduce latent dimensions, but for this dataset, performance improvement was marginal with greater variability in cluster quality.
    * Domain expert(s) should review the 3-Cluster report to apply labels onto the dataset; these labels then can be further leveraged in supervised learning (see below).
* In collaboration with domain expert(s):
    1. Further feature engineer the dataset, i.e. the vehicles' model using 'car name' to enhance clustering by car_brand, year, and car_model;  performance features (i.e. horespower) should be reviewed and validated as reliable basis for classifying "Vintage" status (see below). 
    2. Further iterations in unsupervised learning may be pursued, exploring the use of multi-model ensemble learning.
    3. Supervised learning can be used in lieu of unsupervised learning if Vintage Classification relies heavily on domain expert labelling. 
    4. Use of Vintage Classification can be further discussed as it pertains to binary (i.e. Vintage vs Non-Vintage) versus multiple classes (i.e. Classic vs Retro vs Modern vs Emerging Vintage). 
    5. Vintage Classification can be explored as a continuous score using a regression-based approach to combine weighted factors. 
    6. The model can be leveraged to target audience segmentation: Group potential buyers by their affinity for specific vintage categories. 
    7. The model can be leveraged for marketing insights: Identify which features drive perceptions of vintage value.

In [ ]:
import plotly.express as px
from sklearn.manifold import TSNE
import pandas as pd
import warnings

# Suppress the FutureWarning from Plotly
warnings.filterwarnings("ignore", message="When grouping with a length-1 list-like")

# Decode 'car_brand_encoded' back to original 'car_brand'
df['car_brand'] = le.inverse_transform(df['car_brand_encoded'])

# t-SNE for Core Features
tsne_core = TSNE(n_components=2, random_state=42, perplexity=30)
features_tsne_core = tsne_core.fit_transform(features_core)
tsne_df_core = pd.DataFrame(features_tsne_core, columns=['t-SNE 1', 't-SNE 2'])
tsne_df_core['car brand'] = df.loc[features_core.index, 'car_brand']
tsne_df_core['cluster'] = labels_core
tsne_df_core['model'] = 'Core Features'
tsne_df_core['index'] = features_core.index
tsne_df_core['car name'] = df.loc[features_core.index, 'car name']
tsne_df_core['model year'] = df.loc[features_core.index, 'model year']

# Plot for Core Features only
fig = px.scatter(tsne_df_core, 
                 x='t-SNE 1', 
                 y='t-SNE 2', 
                 color='cluster', 
                 title='Vehicle by Clusters')

# Customize hovertemplate
fig.update_traces(
    hovertemplate='<br>Car Brand: %{customdata[0]}<br>Car Name: %{customdata[1]}<br>Model Year: %{customdata[2]}<br>Cluster: %{customdata[3]}<extra></extra>',
    customdata=tsne_df_core[['car brand', 'car name', 'model year', 'cluster']].to_numpy()
)

# Suppress elements
fig.for_each_annotation(lambda a: a.update(text=''))
fig.update_xaxes(title='', showticklabels=False, showgrid=False, zeroline=False)
fig.update_yaxes(title='', showticklabels=False, showgrid=False, zeroline=False)
fig.update_layout(autosize=False, width=800, height=400, showlegend=False, coloraxis_showscale=False)

# Show the plot
fig.show()


In [ ]:
# Filter and Sort results

import pandas as pd
import ipywidgets as widgets
from IPython.display import display

# Merge tsne_df_core with original df on 'index'
merged_df = tsne_df_core.merge(
    df[['index', 'mpg', 'cylinders', 'displacement', 'horsepower', 'weight', 'acceleration']],
    on='index',
    how='left'
)

# Filter out unwanted columns
data_table_df = merged_df.drop(columns=['t-SNE 1', 't-SNE 2', 'model', 'index'])

# Cast numeric columns to integers
filtered_data = data_table_df.copy()
numeric_columns = ['cluster', 'model year', 'mpg', 'cylinders', 'displacement', 'horsepower', 'weight', 'acceleration']
filtered_data[numeric_columns] = filtered_data[numeric_columns].astype(int)

# Dropdown for sorting columns
sort_column_dropdown = widgets.Dropdown(
    options=filtered_data.columns.tolist(),
    description='Sort by:',
    value=filtered_data.columns[0]
)

# Dropdown for sort order
sort_order_dropdown = widgets.Dropdown(
    options=['Ascending', 'Descending'],
    description='Order:',
    value='Ascending'
)

# Filters for numeric columns with extended value display width
filters = {}
for column in numeric_columns:
    min_val, max_val = filtered_data[column].min(), filtered_data[column].max()
    filters[column] = widgets.IntRangeSlider(
        value=[min_val, max_val],
        min=min_val,
        max=max_val,
        step=1,
        description=column,
        continuous_update=False,
        layout=widgets.Layout(width='600px')  # Extend slider width
    )

# Button to apply filters and sorting
apply_button = widgets.Button(
    description='Apply Filters & Sort',
    button_style='success'
)

# Output widget to display the filtered table
output = widgets.Output()

# Function to update the table based on filters and sorting
def update_table(_):
    df = filtered_data.copy()
    # Apply numeric filters
    for column, slider in filters.items():
        df = df[(df[column] >= slider.value[0]) & (df[column] <= slider.value[1])]
    # Apply sorting
    sort_order = True if sort_order_dropdown.value == 'Ascending' else False
    df = df.sort_values(by=sort_column_dropdown.value, ascending=sort_order)
    # Display the table
    with output:
        output.clear_output(wait=True)
        display(df)

# Attach the function to the apply button
apply_button.on_click(update_table)

# Display all widgets and the output
widgets_ui = widgets.VBox(
    [sort_column_dropdown, sort_order_dropdown, *filters.values(), apply_button, output]
)
display(widgets_ui)
